# LSTM et génération de texte

Ce tutoriel présente l'utilisation d'un LSTM (un type particulier de réseau de neurones récurrents RNN) pour générer du texte.

In [ ]:
#Imports
import numpy as np
import re
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import Adam

## Préparation des données

L'objectif étant de générer des mots (et non des caractères, comme on retrouve parfois dans la littérature), on va découper notre texte grâce à une expression régulière pour récupérer tous les mots. 

On met également le texte en minuscule car les majuscules n'apportent (quasiment) aucune information. 

On pourrait également enlever les apostrophes ou autres signes de ponctuation pour améliorer le résultat final, et certains mots de liaisons/nombres...

In [ ]:
### Import du texte
file_name = "la-machine-a-explorer-le-temps--h-g-wells.txt"
with open(file_name, "r", encoding="utf-8") as f:
    text = f.read()

words = re.findall(r"[\w]+", text.lower())

### Mapping entre les mots et des clés

Les mots n'étant pas compréhensibles par le réseau de neurones, on les converti en nombres pour qu'il puisse travailler efficacement avec (moins de mémoire est nécessaire). On enlève également les doublons, car ils n'ont aucun intérêt.

In [ ]:
### Construction du mapping mot <-> nombre
#On trie par ordre alphabétique notre liste de mots en enlevant les doublons
reduced_words = list(sorted(set(words)))
#Puis on associe une clé à chaque mot (qui est sa position dans la liste)
mapping_words = {word: index for index, word in enumerate(reduced_words)}
#Nombre de mots différents dans le texte
nb_of_words = len(reduced_words)  

### Construction des données d'apprentissage

Ensuite on découpe notre texte en données d'apprentissage, qui sont des couples ("N mots précédents", "mot à prédire")

In [ ]:
### Construction des données d'apprentissage
memory = 2
previous_words = []
next_word = []
for i in range(0, len(words) - memory):
    #On ajoute les mots qui précendent l'index
    previous_words.append(words[i: i + memory])
    #Et dans une autre liste on stock la prédiction associée
    next_word.append(words[i + memory])

nb_learning_data = len(previous_words)

On construit alors deux vecteurs qui regroupent toutes ces données d'apprentissage (X pour la première partie du couple et y pour la seconde). 

Les vecteurs sont des vecteurs numpy, qui seront donnés à Keras pour entraîner le LSTM.

In [ ]:
#On met le tout dans des vecteurs qui seront donnés à Keras
#Remarque : il faut de la mémoire sur votre ordinateur
#On initialise les vecteurs
X = np.zeros((nb_learning_data, memory, nb_of_words), dtype=np.bool)
y = np.zeros((nb_learning_data, nb_of_words), dtype=np.bool)
#On alimente les vecteurs
for i, sentence in enumerate(previous_words):
    for t, word in enumerate(sentence):
        X[i, t, mapping_words[word]] = 1
    y[i, mapping_words[next_word[i]]] = 1

### Déclaration du modèle Keras de réseau de neurones récurrents (de type LSTM)

On va utiliser un LSTM pour faire de la génération de texte. Un certain nombre de paramètres doit être précisé dont :
- la taille du réseau
- le nombre de couples que l'on va traiter en même temps pour entraîner le réseau (généralement, on ne fait pas "un couple" = "une mise à jour du réseau pour l'améliorer", car il oscille trop)
- le nombre de fois que l'on va répéter la procédure d'apprentissage (idéalement 30 fois, si vous avez le temps)
- la vitesse à laquelle le réseau va apprendre (si vous mettez un nombre trop élevé, il pourrait ne jamais atteindre la meilleure prédiction)

Ensuite, on déclarera grâce à Keras notre réseau.

In [ ]:
### Modèle KERAS : un RNN particulier, le LSTM (long short term memory)
# Paramètres
rnn_size = 128
batch_size = 30
num_epochs = 1
learning_rate = 0.001

In [ ]:
# Modèle
model = Sequential()
model.add(LSTM(rnn_size, input_shape=(memory, nb_of_words)))
model.add(Dense(nb_of_words))
model.add(Activation('softmax'))

In [ ]:
optimizer = Adam(lr=learning_rate)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [ ]:
#Entraînement de l'algorithme
model.fit(X, y,batch_size=batch_size,epochs=num_epochs)

In [ ]:
#Sauvegarde du LSTM entraîné
model.save('lstm_text_generation.h5')

## Prédictions de l'algorithme

A l'image de ce qui a été fait avec Markov, on va prédire les mots suivants du début de notre phrase.

In [ ]:
### Prédictions
#Variables
start_sentence = "L'intelligence artificielle"
nb_predictions = 10
prediction = start_sentence
#Initialisation de la mémoire
last_words = re.findall(r"[\w]+", start_sentence.lower())
last_words = last_words[-memory:]

On répète ce qui a été fait en termes de préparation des données, puis on génère le mot suivant de la phrase. Puisque notre algorithme a appris à prédire des nombres, il faut convertir son résultat en "mot" grâce au dictionnaire de mapping construit précédemment.

Ensuite on met à jour la mémoire, pour prédire le mot d'après.

In [ ]:
#On boucle sur le nombre de prédictions à faire pour répéter la procédure indiquée
for i in range(nb_predictions):
    #Construction du vecteur d'entrée du modèle Keras
    x = np.zeros((1, memory, nb_of_words))
    for t, word in enumerate(last_words):
        #Les mots sont convertis en nombres
        x[0, t, mapping_words[word]] = 1.
    #Prédictions du modèle : on a la probabilité de chaque mot du dictionnaire d'être le suivant
    possibilities = model.predict(x, verbose=0)[0]  
    #On prend le mot avec la probabilité la plus élevée
    #En général, on préfère utiliser une notion de "température" pour laisser une part au hasard
    key_word = np.argmax(possibilities)
    #On traduit le nombre obtenu en mot grâce au dictionnaire
    #Pour comprendre le code : on converti les clés du dictionnaire en une liste et pareil pour les valeurs
    #Ensuite, on prend dans le dictionnaire des clés l'entrée qui a le même numéro que la valeur voulue
    #Car rappelons-le, notre dictionnaire est de la forme "mot":nombre et nous on a un nombre là
    pred = list(mapping_words.keys())[list(mapping_words.values()).index(key_word)]
    
    #On ajoute le mot prédit à la phrase
    prediction += " "+pred
    #Et on met à jour la mémoire
    last_words.pop(0)
    last_words.append(pred)

On affiche le résultat final

In [ ]:
print(prediction)